In [1]:
import numpy as np
import matplotlib.pyplot as plt
import math
import random
import time
from dataset_PA1.dataloader import Dataloader
from dataset_PA1.dataloader import datasetIterator
from CNN_classes_2 import Conv2D
from CNN_classes_2 import ReLU
from CNN_classes_2 import MaxPool2D
from CNN_classes_2 import Flatten
from CNN_classes_2 import Linear
from CNN_classes_2 import SoftmaxCrossEntropy

random.seed(42)

In [2]:
conv1 = Conv2D(1, 8, 3, lr=0.01)
relu1 = ReLU()
pool1 = MaxPool2D()
conv2 = Conv2D(8, 16, 4, lr=0.01)
relu2 = ReLU()
pool2 = MaxPool2D()
flatten = Flatten()
fc = Linear(16 * 5 * 5, 10, lr=0.01)
criterion = SoftmaxCrossEntropy()

In [3]:
data_path = './dataset_PA1'
learning_rate = 0.01
epochs = 5
batch_size = 32
train_loader = Dataloader(path=data_path, is_train=True, batch_size=batch_size)
test_loader = Dataloader(path=data_path, is_train=False, batch_size=batch_size)

In [ ]:
x_train = train_loader.images
y_train = train_loader.labels
x_test = test_loader.images
y_test = test_loader.labels

y_test_label = np.argmax(y_test, axis=1)

train_loss_history = []
train_acc_history = []
test_acc_history = []

def forward_pass(x):
    out = conv1.forward(x)
    out = relu1.forward(out)
    out = pool1.forward(out)
    out = conv2.forward(out)
    out = relu2.forward(out)
    out = pool2.forward(out)
    out = flatten.forward(out)
    out = fc.forward(out)
    return out

for epoch in range(epochs):
    perm = np.random.permutation(len(x_train))
    x_train, y_train = x_train[perm], y_train[perm]
    total_loss = 0
    correct = 0
    n_batches = len(x_train) // batch_size
    time_start = time.time()
    print(f"Epoch {epoch+1}/{epochs} started.")
    for i in range(0, len(x_train), batch_size):
        xb = x_train[i:i+batch_size]
        yb = y_train[i:i+batch_size]

        # forward
        out = forward_pass(xb)
        loss = criterion.forward(out, yb)

        preds = np.argmax(out, axis=1)
        labels = np.argmax(yb, axis=1)
        correct += np.sum(preds == labels)
        total_loss += loss

        # backward
        d_out = criterion.backward()
        d_out = fc.backward(d_out)
        d_out = flatten.backward(d_out)
        d_out = pool2.backward(d_out)
        d_out = relu2.backward(d_out)
        d_out = conv2.backward(d_out)
        d_out = pool1.backward(d_out)
        d_out = relu1.backward(d_out)
        _ = conv1.backward(d_out)

    train_acc = correct / len(x_train)
    avg_loss = total_loss / n_batches
    train_loss_history.append(avg_loss)
    train_acc_history.append(train_acc)

    # --- 테스트셋 평가 ---
    test_preds = []
    test_preds = []
    for j in range(0, len(x_test), batch_size):
        xt = x_test[j:j+batch_size]
        logits = forward_pass(xt)
        exp_scores = np.exp(logits - np.max(logits, axis=1, keepdims=True))
        probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)
        test_preds.append(np.argmax(probs, axis=1))
    test_preds = np.concatenate(test_preds)
    test_acc = np.mean(test_preds == y_test_label)
    test_acc_history.append(test_acc)

    print(f"Epoch {epoch+1}/{epochs} | Loss: {avg_loss:.4f} | Train Acc: {train_acc:.4f} | Test Acc: {test_acc:.4f} | Time: {time.time()-time_start}")


Epoch 1/5 started.


In [ ]:
epochs_range = np.arange(1, epochs + 1)

fig, ax1 = plt.subplots(figsize=(8,5))
ax2 = ax1.twinx()

# Loss curve
ax1.plot(epochs_range, train_loss_history, 'o-', color='tab:red', label='Train Loss')
ax1.set_xlabel("Epoch")
ax1.set_ylabel("Loss", color='tab:red')
ax1.tick_params(axis='y', labelcolor='tab:red')
ax1.grid(True, linestyle='--', alpha=0.4)

# Accuracy curve
ax2.plot(epochs_range, train_acc_history, 's--', color='tab:blue', label='Train Acc')
ax2.plot(epochs_range, test_acc_history, 'd-', color='tab:green', label='Test Acc')
ax2.set_ylabel("Accuracy", color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')

# Combined legend
lines, labels = ax1.get_legend_handles_labels()
lines2, labels2 = ax2.get_legend_handles_labels()
ax2.legend(lines + lines2, labels + labels2, loc='center right')

plt.title("Training Loss & Accuracy Curve")
plt.tight_layout()
plt.savefig("training_curve.png")
plt.show()

In [ ]:
num_classes = 10
cm = np.zeros((num_classes, num_classes), dtype=int)

for t, p in zip(y_test_label, test_preds):
    cm[t, p] += 1

# --- 시각화 ---
fig, ax = plt.subplots(figsize=(6,6))
im = ax.imshow(cm, cmap='Blues')
plt.colorbar(im, ax=ax)

ax.set_xlabel("Predicted Label")
ax.set_ylabel("True Label")
ax.set_title("Confusion Matrix (Test Set)")
ax.set_xticks(range(num_classes))
ax.set_yticks(range(num_classes))
ax.set_xticklabels(range(num_classes))
ax.set_yticklabels(range(num_classes))

# 각 칸에 숫자 표시
for i in range(num_classes):
    for j in range(num_classes):
        text_color = "white" if cm[i, j] > cm.max() * 0.5 else "black"
        ax.text(j, i, str(cm[i, j]), ha='center', va='center', color=text_color, fontsize=8)

plt.tight_layout()
plt.savefig("confusion_matrix_numpy.png")
plt.show()

In [ ]:
num_classes = 10
top_k = 3

# 테스트셋 다시 forward (softmax 확률 포함)
test_probs = []
test_labels = []
for j in range(0, len(x_test), batch_size):
    xt = x_test[j:j+batch_size]
    out = forward_pass(xt)  # softmax까지 포함된 확률
    test_probs.append(out)
test_probs = np.concatenate(test_probs, axis=0)

# 각 클래스별 top3 index 추출
top_images = []  # [(img, pred, prob), ...]
for c in range(num_classes):
    # class c에 대해 예측 확률의 c번째 column을 가져오기
    class_confidences = test_probs[:, c]
    top_indices = np.argsort(class_confidences)[-top_k:][::-1]
    for idx in top_indices:
        top_images.append((x_test[idx, 0], np.argmax(test_probs[idx]), class_confidences[idx]))

# --- 시각화 ---
fig, axes = plt.subplots(num_classes, top_k, figsize=(6, 10))
fig.suptitle("Top-3 Confident Predictions per Class", fontsize=14)

for c in range(num_classes):
    class_confidences = test_probs[:, c]
    top_indices = np.argsort(class_confidences)[-top_k:][::-1]
    for k, idx in enumerate(top_indices):
        ax = axes[c, k]
        ax.imshow(x_test[idx, 0], cmap='gray')
        ax.axis('off')
        pred_label = np.argmax(test_probs[idx])
        prob = test_probs[idx, pred_label]
        ax.set_title(f"P:{pred_label} ({prob*100:.1f}%)", fontsize=7)

plt.tight_layout(rect=[0, 0, 1, 0.97])
plt.savefig("top3_predictions.png")
plt.show()